In [ ]:
project = 'saga-data'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

## Eksempel 1
Jeg har fått noen data med såkalt "vegsystemreferanse kortform", hvor i all verden er dette egentlig?

In [ ]:
query = f"""
with eksempeldata as (
  select "0.01587524-0.2725013@971774" as kortform, 42 as measurement
)

, eksempeldata_brutt_opp_steg1 as (
  select
  SPLIT(kortform, '@')[OFFSET(0)] as start_til_sluttposisjon
  ,SPLIT(kortform, '@')[OFFSET(1)] as veglenkesekvensid
  from eksempeldata
)

, eksempeldata_brutt_opp_steg2 as (
  select 
  cast(veglenkesekvensid as int64) as veglenkesekvensid
  , cast(SPLIT(start_til_sluttposisjon, '-')[OFFSET(0)] as float64) as startposisjon
  , cast(SPLIT(start_til_sluttposisjon, '-')[OFFSET(1)] as float64) as sluttposisjon
  from eksempeldata_brutt_opp_steg1
)

, geomaster as (
  select * from saga-data.geomaster.geomaster
  where run_yearmonth = "202305"
)


select vegkategori, vegnummer, kommunenavn, sum(lengde_til_neste_punkt) as antall_meter 
from eksempeldata_brutt_opp_steg2 as a inner join geomaster as b
on a.veglenkesekvensid = b.veglenkesekvensid
and b.punkt_approx_relative_pos between a.startposisjon and a.sluttposisjon
group by 1,2,3
order by 1,2,3
"""

print(query)

client.query(query).to_dataframe()

Svar: Denne er på E6 i Halden, og er ca 1.8km lang.

## Eksempel 2
Jeg har to datasett som kun er geografiske objekter men ikke noe mer info. Hvordan kan jeg koble de sammen der de overlapper?

In [ ]:
query = f"""
with eksempeldata1 as (
select 
172 as id
, ST_GeogFromText('LINESTRING(10.227466 59.723153, 10.22756 59.723072, 10.22765 59.722985, 10.227688 59.722949, 10.227723 59.722912, 10.227759 59.722874, 10.227794 59.722838, 10.227828 59.7228, 10.227862 59.722763, 10.227896 59.722725, 10.227928 59.722688, 10.22796 59.722649, 10.22799 59.722612)') as geography
, 42 as measurement_one
)

, eksempeldata2 as (
select 
172 as id
, ST_GeogFromText('LINESTRING(10.227688 59.722949, 10.227723 59.722912, 10.227759 59.722874, 10.227794 59.722838)') as geography
, 78 as measurement_two
)

, geomaster as (
  select * from saga-data.geomaster.geomaster
  where run_yearmonth = "202305"
)

select
a.*, b.measurement_one, c.measurement_two
from 
geomaster as a
left join eksempeldata1 as b
on st_intersects(st_buffer(b.geography,10), a.punkt_latlon)
left join eksempeldata2 as c
on st_intersects(st_buffer(c.geography,10), a.punkt_latlon)
where measurement_one is not null or measurement_two is not null
order by veglenke_referanse, punktrekkefolge asc
"""

print(query)

client.query(query).to_dataframe()

Vi ser de overlapper omtrent i midten, og får ut masse nyttig info om punktene de treffer.

## Eksempel 3
Jeg vil ha geografiske linestrings for europavei, ca mellom Trondheim og Oppdal.

In [ ]:
query = f"""
with geomaster as (
  select * from saga-data.geomaster.geomaster
  where vegkategori = "E"
  and vegnummer = 6
  and veglenketype = "normal_strekning"
  and run_yearmonth = "202305"
   )

select 
veglenkesekvensid
, veglenke_referanse
, st_makeline(array_agg(punkt_latlon)) as linestrings
from
(
select veglenkesekvensid
, veglenke_referanse
, punktrekkefolge
, punkt_latlon
from geomaster
where grader_nord > (select avg(grader_nord) from geomaster where kommunenavn = "Oppdal")
and grader_nord < (select avg(grader_nord) from geomaster where kommunenavn = "Trondheim")
order by 1,2,3 asc
)
group by 1,2
LIMIT 20 -- fjern denne for å hente alle
"""

print(query)

client.query(query).to_dataframe()


Se på linestringene i GeoViz for å se om de virker OK.

## Eksempel 4
Hvilken kommune har størst andel sideanlegg over omtrentlig totallengde riksvei?

In [ ]:
query = f"""
select 
kommunenavn,
sum(case when veglenketype = "sideanlegg" then lengde_til_neste_punkt else 0 end)/
sum(case when veglenketype is not null then lengde_til_neste_punkt else 0 end) as andel_sideanlegg
from saga-data.geomaster.geomaster
where vegkategori = "R"
and run_yearmonth = "202305"
group by kommunenavn
order by andel_sideanlegg desc
"""

print(query)

client.query(query).to_dataframe()

Vi får sortert ut den kommunen med størst relativ andell sideanlegg øverst.

## Eksempel 5
Sjekk andelen trafikklenker på Europavei fra Nordland og nordover som har en rasteplass eller dognhvilkeplass et sted på lenken.

In [ ]:
query = f"""
with rasteplass as (
select 1 as rasteplass, lokasjon.kommuner, ls.veglenkesekvensid, ls.startposisjon, ls.sluttposisjon from `saga-nvdb-prod-vlmh.standardized.vegobjekter_rasteplass`
cross join unnest(lokasjon.stedfestinger) as ls
where metadata.sluttdato is null
)

, dognhvileplass as (
select 1 as dognhvileplass, lokasjon.kommuner, ls.veglenkesekvensid, ls.startposisjon, ls.sluttposisjon from `saga-nvdb-prod-vlmh.standardized.vegobjekter_dognhvileplass`
cross join unnest(lokasjon.stedfestinger) as ls
where metadata.sluttdato is null
)

, geomaster as (
  select * from saga-data.geomaster.geomaster 
  where vegkategori = "E"
  and run_yearmonth = "202305"
  and grader_nord > (select min(grader_nord) from saga-data.geomaster.geomaster where fylkesnavn = "Nordland")
)

, joined as (
select a.*, coalesce(b.rasteplass,0) as rasteplass, coalesce(c.dognhvileplass,0) as dognhvileplass 

from geomaster  as a

left join rasteplass as b 
on a.veglenkesekvensid = b.veglenkesekvensid and a.punkt_approx_relative_pos between b.startposisjon and b.sluttposisjon

left join dognhvileplass as c
on a.veglenkesekvensid = c.veglenkesekvensid and a.punkt_approx_relative_pos between c.startposisjon and c.sluttposisjon
)

, har_eller_har_ikke as (
select trafikklenkeid
, max(case when (rasteplass + dognhvileplass) > 0 then 1 else 0 end) as raste_eller_dognhvileplass
from joined
group by 1
)

select avg(b.raste_eller_dognhvileplass) 
from 
joined as a 
inner join 
har_eller_har_ikke as b
on  
a.trafikklenkeid = b.trafikklenkeid
"""

print(query)

client.query(query).to_dataframe()

Vi ser hvilken andel trafikklenker som er tilknyttet raste eller døgnhvileplass

## Eksempel 6
Hvilken kommune og veitype hadde flest registrert asfaltarbeider i 2022?

**NB:** vil dobbeltelle i de tilfellene der samme asfaltarbeid traff flere kommuner.

In [ ]:
query = f"""
with geomaster as (

  select * from `saga-data.geomaster.geomaster`
  where vegkategori in ("E","R")
  and run_yearmonth = "202305"
)


,paved_road as (

select id, 1 as paved_flag
, ls.veglenkesekvensid
, ls.startposisjon
, ls.sluttposisjon
from `saga-nvdb-prod-vlmh.standardized.vegobjekter_vegdekke`, unnest(lokasjon.stedfestinger) ls
where extract(year from egenskaper.dekkeleggingsdato) = 2022
and metadata.sluttdato is null

)


, joined as (
  
select a.*, b.paved_flag
from
(select * from geomaster) as a
left join 
(select * from paved_road) as b
on a.veglenkesekvensid = b.veglenkesekvensid
and a.punkt_approx_relative_pos between b.startposisjon and b.sluttposisjon
)


select kommunenavn, vegkategori, count(paved_flag) as antall_asfalteringer
from joined
group by 1,2
order by 3 desc
"""

print(query)

client.query(query).to_dataframe()

Vi ser antall asfalteringer sortert på høyest øverst.

## Eksempel 7
Jeg har fått en måling med vegsystemreferanse med bokstaver og meter, kan jeg få et datasett om denne strekningen?

**NB:** dette eksempelet er helt validert til å være helt likt hvordan vegsystemreferansene ser ut, men ta det som eksempel.

In [ ]:
query = f"""
with eksempeldata as (
  select "EV18 S56D1 m7000-8000" as vegsystemreferanse, 42 as measurement
)

, eksempeldata_brutt_opp_steg1 as (
  select
  SPLIT(vegsystemreferanse, ' m')[OFFSET(0)] as vegsystem_ref_delvis
  ,SPLIT(vegsystemreferanse, ' m')[OFFSET(1)] as vegsystem_ref_meter_urenset
  , measurement
  from eksempeldata
)

, eksempeldata_brutt_opp_steg2 as (
  select
  vegsystem_ref_delvis
  , cast(SPLIT(vegsystem_ref_meter_urenset, '-')[OFFSET(0)]  as float64) as vegsystem_meter_fra
  , cast(SPLIT(vegsystem_ref_meter_urenset, '-')[OFFSET(1)] as float64) vegsystem_meter_til
  , measurement
  from eksempeldata_brutt_opp_steg1
)


, geomaster as (
  select * except(vegsystem_ref_delvis)
  ,  SPLIT(vegsystem_ref_delvis, ' m')[OFFSET(0)] as vegsystem_ref_delvis --dette er en liten bug som skal fikses
  from saga-data.geomaster.geomaster
  where run_yearmonth = "202305"
)


select b.*, a.measurement
from eksempeldata_brutt_opp_steg2 as a inner join geomaster as b
on a.vegsystem_ref_delvis = b.vegsystem_ref_delvis
and b.vegsystem_ref_meter between a.vegsystem_meter_fra and a.vegsystem_meter_til
"""

print(query)

client.query(query).to_dataframe()

Vipps, et datasett med mye mer info.

